In [ ]:
%pip install llama-index llama-index-graph-stores-neo4j openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.7 MB/s eta 0:00:00


In [ ]:
# this version is on spiritual texts,  Vidura Neethi,  Bhagavad Gita, , Ashtavakra Sanksrit, Sri Aurobindo
from llama_index.core import SimpleDirectoryReader
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
import os
import openai
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor
import nest_asyncio
import asyncio

# Apply nest_asyncio
nest_asyncio.apply()

os.environ["OPENAI_API_KEY"] = "your_api_key_here"
openai.api_key = os.environ["OPENAI_API_KEY"]

from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore


graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password="your_neo4j_instance_password",
    url="your_neo4j_db.databases.neo4j.io",
)

# Use the existing index

index = PropertyGraphIndex.from_existing(
    property_graph_store=graph_store,
    llm=OpenAI(model="gpt-4-turbo", temperature=0.7),
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
)





In [ ]:
# # Define retriever- default one
# retriever = index.as_retriever(
#     include_text=False,  # include source text in returned nodes, default True
# )

In [ ]:
# Define custom retriever-from github - 18 July 6:44am
from pydantic import BaseModel
from typing import Optional, List


class Entities(BaseModel):
    """List of named entities in the text such as names of people, organizations, concepts, and spiritual texts"""
    names: Optional[List[str]]


prompt_template_entities = """
Extract all named entities such as names of people, organizations, concepts, and spiritual texts
from the following text:
{text}
"""

In [ ]:
# Define custom retriever-from github - 18 July 6:44am
from typing import Any, Optional

from llama_index.core.embeddings import BaseEmbedding
from llama_index.core.retrievers import CustomPGRetriever, VectorContextRetriever
from llama_index.core.vector_stores.types import VectorStore
from llama_index.program.openai import OpenAIPydanticProgram


class MyCustomRetriever(CustomPGRetriever):
    """Custom retriever with cohere reranking."""

    def init(
        self,
        ## vector context retriever params
        embed_model: Optional[BaseEmbedding] = None,
        vector_store: Optional[VectorStore] = None,
        similarity_top_k: int = 4,
        path_depth: int = 1,
        include_text: bool = True,
        **kwargs: Any,
    ) -> None:
        """Uses any kwargs passed in from class constructor."""
        self.entity_extraction = OpenAIPydanticProgram.from_defaults(
            output_cls=Entities, prompt_template_str=prompt_template_entities
        )
        self.vector_retriever = VectorContextRetriever(
            self.graph_store,
            include_text=self.include_text,
            embed_model=embed_model,
            similarity_top_k=similarity_top_k,
            path_depth=path_depth,
        )

    def custom_retrieve(self, query_str: str) -> str:
        """Define custom retriever with reranking.

        Could return `str`, `TextNode`, `NodeWithScore`, or a list of those.
        """
        entities = self.entity_extraction(text=query_str).names
        result_nodes = []
        if entities:
            print(f"Detected entities: {entities}")
            for entity in entities:
                result_nodes.extend(self.vector_retriever.retrieve(entity))
        else:
            result_nodes.extend(self.vector_retriever.retrieve(query_str))
        ## TMP: please change
        final_text = "\n\n".join(
            [n.get_content(metadata_mode="llm") for n in result_nodes]
        )
        return final_text

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.7)
#llm = OpenAI(model="gpt-4o", temperature=0.0)
embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")
from llama_index.core.query_engine import RetrieverQueryEngine

custom_sub_retriever = MyCustomRetriever(
    index.property_graph_store,
    include_text=True,
    vector_store=index.vector_store,
    embed_model=embed_model
)

query_engine = RetrieverQueryEngine.from_args(
    index.as_retriever(sub_retrievers=[custom_sub_retriever]), llm=llm
)

In [ ]:
response = query_engine.query("What are the 3 most important practical teachings that one can imbibe in daily life from Vidura and mention source?")
print(str(response))

Forgiveness, Non-violence, and Truth are the three most important practical teachings that one can imbibe in daily life from Vidura. The source of these teachings can be found in the work Vidura worked on, namely Vidura-N∂ti.


In [ ]:
# results = retriever.retrieve("What are the 3 most important practical teachings that one can imbibe in daily life from Vidura?")
# for record in results:
#     print(record.text)
# query_engine = index.as_query_engine(include_text=True)
# response = query_engine.query("What are the 3 most important practical teachings that one can imbibe in daily life from Vidura?")
# print(str(response))

Vidura -> WORKED_ON -> Vidura-N∂ti
Vidura -> WORKED_ON -> Udyogaparvaƒi-Prajågaraparvaƒi
Vidura -> WORKED_ON -> Mahåbhårata
Vidura -> WORKED_ON -> Neeti
Vidura -> WORKED_ON -> King's Chambers
Vidura -> WORKED_ON -> Human welfare
Vidura -> WORKED_ON -> Friendship
Vidura -> WORKED_ON -> Forgiveness
Vidura -> WORKED_ON -> Non-violence
Vidura -> WORKED_ON -> Justice
Vidura -> WORKED_ON -> Truth
Vidura -> WORKED_ON -> Polity
Vidura -> WORKED_ON -> Råja Dharma
Vidura -> WORKED_ON -> Mahåbhårata-Udyogaparva
Virochana -> WORKED_ON -> spiritual instructions from Prajapati
Ashtavakra -> WORKED_ON -> Ashtavakra-Gita
Friendship, Forgiveness, and Non-violence.


In [ ]:
response = query_engine.query("How can one correlate Ashtavakra Gita, Vidura Neeti and Bhagavad Gita?")
print(str(response))

One can correlate Ashtavakra Gita, Vidura Neeti, and Bhagavad Gita by recognizing their teachings on aspects such as liberation, wisdom, self-realization, and the nature of reality. Each text provides insights into the path to liberation, the importance of detachment, the nature of the self, and the significance of wisdom in navigating life's challenges. By studying these texts, one can gain a deeper understanding of spiritual principles and practical guidance for leading a fulfilling and purposeful life.


In [ ]:
response = query_engine.query("Correlate Ashtavakra Gita, Vidura Neeti and Bhagavad Gita and generate a two stanza poem")
print(str(response))

In teachings profound, Ashtavakra spoke,  
Mind's bondage and liberation he awoke.  
Transcending desires, attachment's snare,  
In detachment lies freedom, beyond all care.  

Vidura's wisdom, like a guiding light,  
Echoes Bhagavad Gita's truth so bright.  
Through selfless devotion and wisdom pure,  
Attain the Supreme, salvation's ultimate lure.
